# get f1score, recall, precision


extract emojis from the column 'emoji_sentence', named as 'real_emojis',  

use the columns 'real_emojis' and 'predicted_emojis' to get precision, recall, f1score between them.

for specific model, such as mistral, qwen, llama, there are some text in the column 'predicted_emojis', we need to remove text to get predicted_pure_emojis, then compute precision recall f1score between 'real_emojis' and 'predicted_pure_emojis'.




In [1]:
import pandas as pd
import re

import os
from together import Together

from openai import OpenAI
import random
from collections import Counter

import math

In [2]:
prompt_augmentation = False

In [110]:
model_name='gpt-4o'
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
# model_name = 'qwen2-72b-instruct'

# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)

gpt-4o
gpt-4o-mini


In [111]:
dataset_path = f'./dataset_with_logits/{model_name}/prompt_aug_sentence_cls/stance/dataset_logits_{model_name}_prompt_aug_sentence_cls_stance_5039.csv'

# dataset_path = f'./dataset_finalOutput/random_sentence/dataset_labels_using_random_sentence_stance_5039.csv'
print(dataset_path)

emoji_dataset = pd.read_csv(dataset_path)
dataset_len = len(emoji_dataset)
print(dataset_len)

./dataset_with_logits/gpt-4o/prompt_aug_sentence_cls/stance/dataset_logits_gpt-4o_prompt_aug_sentence_cls_stance_5039.csv
5039


In [112]:

augmentation_method = 'prompt_aug_sentence_cls'
# augmentation_method = 'conditional_recommend'
# augmentation_method = 'fewshot'
augmentation_bool=True
print(augmentation_bool)
print(augmentation_method)
print(model_name)

True
prompt_aug_sentence_cls
gpt-4o


In [88]:
!ls ./condition_recommed_fewshot/dataset_logits_gemma2-9b-instruct_fewshot_stance_5039.csv



./condition_recommed_fewshot/dataset_logits_gemma2-9b-instruct_fewshot_stance_5039.csv


In [89]:

dataset_path = './condition_recommed_fewshot/dataset_logits_gemma2-9b-instruct_fewshot_stance_5039.csv'

emoji_dataset = pd.read_csv(dataset_path)
dataset_len=len(emoji_dataset)

In [90]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  5039 non-null   int64 
 1   index                       5039 non-null   int64 
 2   user_id                     5039 non-null   object
 3   emoji_sentence              5039 non-null   object
 4   char_count                  5039 non-null   int64 
 5   cluster                     5039 non-null   int64 
 6   sentence                    5039 non-null   object
 7   gender_labels               5039 non-null   object
 8   sentiment_labels            5039 non-null   object
 9   emotion_labels              5039 non-null   object
 10  age_labels                  5039 non-null   object
 11  stance_labels               5039 non-null   object
 12  predicted_emojis            5039 non-null   object
 13  predicted_sentence          5039 non-null   obje

In [102]:

def extract_emojis(text):
    if text is None or (isinstance(text, float) and math.isnan(text)):
        return [] 
    emoji_pattern = re.compile(
        "["                              
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F700-\U0001F77F"  # Alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
        u"\U0001F004"             # Mahjong Tile Red Dragon
        u"\U0001F0CF"             # Playing Cards Joker
        u"\U0001F1E6-\U0001F1FF"  # Flags
        u"\U000025AA-\U000025AB"  # Small Squares
        u"\U000025FE-\U000025FF"  # Black Medium Squares
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U00002B50"             # Star
        u"\U0001F1E6-\U0001F1FF"  # Regional indicator symbols
        u"\u2640-\u2642"          # Gender signs
        u"\u2600-\u26FF"          # Miscellaneous Symbols
        u"\u2700-\u27BF"          # Dingbats
        "]+", flags=re.UNICODE)

    return emoji_pattern.findall(text)


In [103]:
columns_list = emoji_dataset.columns.tolist()
print(columns_list)

['Unnamed: 0.1', 'Unnamed: 0', 'index', 'user_id', 'emoji_sentence', 'char_count', 'cluster', 'sentence', 'gender_labels', 'sentiment_labels', 'emotion_labels', 'age_labels', 'stance_labels', 'is_duplicate', 'predicted_emojis', 'predicted_sentence', 'predicted_pure_emojis', 'predicted_pure_sentence', 'gender_logits', 'sentiment_logits', 'emotion_logits', 'age_logits', 'stance_logits', 'real_emojis', 'gender_labels_using_sentence', 'sentiment_labels_using_sentence', 'emotion_labels_using_sentence', 'age_labels_using_sentence', 'stance_labels_using_sentence', 'predicted_emojis_prompt_aug_sentence_cls', 'predicted_sentence_prompt_aug_sentence_cls', 'predicted_pure_emojis_prompt_aug_sentence_cls', 'predicted_pure_sentence_prompt_aug_sentence_cls', 'gender_logits_prompt_aug_sentence_cls', 'sentiment_logits_prompt_aug_sentence_cls', 'emotion_logits_prompt_aug_sentence_cls', 'age_logits_prompt_aug_sentence_cls', 'stance_logits_prompt_aug_sentence_cls']


In [104]:

if 'real_emojis' not in columns_list: 

    for i in range(dataset_len):
        text=emoji_dataset.at[i,'emoji_sentence']
        list_emojis = extract_emojis(text)
        # print(i, list_emojis)
        str_emojis=''.join(list_emojis)
        print(i,str_emojis, len(str_emojis))
    
        emoji_dataset.at[i, 'real_emojis']=str_emojis
    



In [101]:

if 'random_emojis' not in columns_list: 

    for i in range(dataset_len):
        text=emoji_dataset.at[i,'random_sentence']
        list_emojis = extract_emojis(text)
        # print(i, list_emojis)
        str_emojis=''.join(list_emojis)
        print(i,str_emojis, len(str_emojis))
    
        emoji_dataset.at[i, 'random_emojis']=str_emojis
    


0 🦄🐇🌹 3
1 🎶♨🌌 3
2 🏆🕯💞 3
3 ✝🧢🔝 3
4 🚴👅🏆 3
5 🍿🎧🎈 3
6 📌🦊❣ 3
7 🙀🎂🥓 3
8 🌴🦅☎ 3
9 🍀🍎🐢 3
10 👽🍟😑 3
11 🎂💅👺 3
12 😙🕺 2
13 ⚾🌍🗣 3
14 💧👙👉 3
15 🦜😵🧡 3
16 🧐☠ 2
17 🖤🤔🥰 3
18 🥀🍎🦷 3
19 🥰🎄👂 3
20 🛒🌌🍋 3
21 🤡🧢💫 3
22 🎻🌧👇 3
23 💊🎈🌃 3
24 🌱🌮🇨🇦 4
25 💨🏵🥔 3
26 🦀🗿🐢 3
27 😳🛐🍨 3
28 🥓🥇🦖 3
29 🌚⚾👯 3
30 🦍😩😥 3
31 💚🌧🌛 3
32 🛌👗 2
33 🍦☝🐸 3
34 🙋💻 2
35 😌😒🎓 3
36 📱🥰🛐 3
37 🌊🤣👩 3
38 🥀😽⚡ 3
39 ✌👩🦀 3
40 😾🍷🍌 3
41 🎁👆😳 3
42 🤜👑🌲 3
43 👧🤷⛽ 3
44 😮🤭🌅 3
45 💌🐴 2
46 📌👁🙊 3
47 🙈🎊😞 3
48 😋🕳🍋 3
49 😘🖐🍋 3
50 🌴🥴 2
51 🐢🥧🌉 3
52 🌄😠🙄 3
53 💪😫👉 3
54 🍆🍰 2
55 🤩📖🎆 3
56 🔮🌟🤲 3
57 🥩😕😡 3
58 😉🦄🎂 3
59 🔞😻🐓 3
60 🌧🥰👂 3
61 🌿🍋💖 3
62 ☑😹👠 3
63 🥶💣💫 3
64 🧁🎵🥑 3
65 👧🎻😇 3
66 🥃💚🌌 3
67 ⚽🌻🎓 3
68 😲🍅☘ 3
69 🍔💤🎄 3
70 💭🦜🐟 3
71 🌱⚫🏡 3
72 💰♥💪 3
73 😅🥚💯 3
74 👣🧘🐈 3
75 😉🎉🕯 3
76 💤🙁🏅 3
77 🙅😨📩 3
78 😃🥓🍭 3
79 🐢🏡❌ 3
80 🧡🔞🍾 3
81 💚🥕🐠 3
82 ❗💢🌃 3
83 🍴🤭🤘 3
84 💗🤠📩 3
85 🌷🗣🍟 3
86 🐷🐟🙈 3
87 🍃😰🚩 3
88 📍😓👯 3
89 😲📞 2
90 🔵💞👺 3
91 📍🍏💧 3
92 📺🌕🚀 3
93 ❄😾🤝 3
94 👹🤐🧿 3
95 😖😎🔔 3
96 😌🔗🍸 3
97 😿🌳👇 3
98 😋💯💔 3
99 🍀🤓🌵 3
100 🐱🔥🐼 3
101 🍔💎🤤 3
102 😁🌟👁 3
103 👏🌵🇹🇷 4
104 🌾📖🛌 3
105 💤👻👺 3
106 🌲🔵🐸 3
107 ❤📢👏 3
108 🍓🤙💔 3
109 💊💍💘 3
110 😌🥺💥 3
111 🌎🤯

In [35]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 30 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Unnamed: 0                                5039 non-null   int64 
 1   index                                     5039 non-null   int64 
 2   user_id                                   5039 non-null   object
 3   emoji_sentence                            5039 non-null   object
 4   char_count                                5039 non-null   int64 
 5   cluster                                   5039 non-null   int64 
 6   sentence                                  5039 non-null   object
 7   gender_labels                             5039 non-null   object
 8   sentiment_labels                          5039 non-null   object
 9   emotion_labels                            5039 non-null   object
 10  age_labels                                5039 n

In [113]:


# Define a function to compute precision, recall, and F1 score for a single pair of real and predicted emojis
def calculate_metrics(real_emojis, predicted_emojis):
    # real_set = set(real_emojis)
    if real_emojis is None or (isinstance(real_emojis, float) and math.isnan(real_emojis)):
        real_emojis = []
    real_set = set(real_emojis)
    # predicted_set = set(predicted_emojis)
    # Handle NaN or None in predicted_emojis
    if predicted_emojis is None or (isinstance(predicted_emojis, float) and math.isnan(predicted_emojis)):
        predicted_emojis = []
    predicted_set = set(predicted_emojis)

    # True Positives: Correctly predicted emojis
    true_positives = real_set.intersection(predicted_set)

    # Precision: Proportion of predicted emojis that are correct
    precision = len(true_positives) / len(predicted_set) if predicted_set else 0

    # Recall: Proportion of actual emojis that are predicted
    recall = len(true_positives) / len(real_set) if real_set else 0

    # F1 Score: Harmonic mean of precision and recall
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1_score

In [114]:

# prompt_augmentation=False
# print(prompt_augmentation)
# print(model_name)


augmentation_method = 'prompt_aug_sentence_cls'
# augmentation_method = 'conditional_recommend'
# augmentation_method = 'fewshot'
augmentation_bool=True
print(augmentation_bool)
print(augmentation_method)

print(model_name)

True
prompt_aug_sentence_cls
gpt-4o


In [115]:
specific_models = ['mistral-7b-instruct', 'qwen2-72b-instruct']

# Assume your DataFrame is named df
precisions = []
recalls = []
f1_scores = []

for i, row in emoji_dataset.iterrows():
    if i>=dataset_len: break
    # print(i)
    real_emojis = row['real_emojis']
    # print(real_emojis)
    if model_name == 'random_sentence':
        # print(1)
        predicted_emojis = row['random_emojis']
    else: 
        if augmentation_bool==True:
            if model_name in specific_models: 
                col_name = f'predicted_pure_emojis_{augmentation_method}'
                predicted_emojis = row[col_name]
            else: 
                col_name = f'predicted_emojis_{augmentation_method}'
                predicted_emojis = row[col_name]
        else:
            if model_name in specific_models: 
                predicted_emojis = row['predicted_pure_emojis']
            else: 
                predicted_emojis = row['predicted_emojis']
    
    precision, recall, f1_score = calculate_metrics(real_emojis, predicted_emojis)
    
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)

# Calculate the average metrics across all rows
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)
avg_f1_score = sum(f1_scores) / len(f1_scores)

print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1 Score: {avg_f1_score:.4f}")


Precision: 0.0480
Recall: 0.1295
F1 Score: 0.0653


In [116]:
folder_path = f"./f1score_output"
os.makedirs(folder_path, exist_ok=True)

In [117]:
# Save the results to a text file named after the model
if augmentation_bool==True: 
    folder_path = f'./f1score_output/{augmentation_method}'
    output_filename = f"./f1score_output/{augmentation_method}/{model_name}_{augmentation_method}_f1score_{dataset_len}.txt"
else: 
    folder_path = f'./f1score_output'
    output_filename = f"./f1score_output/{model_name}_f1score_{dataset_len}.txt"
os.makedirs(folder_path, exist_ok=True)

with open(output_filename, "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f"Precision: {avg_precision:.4f}\n")
    f.write(f"Recall: {avg_recall:.4f}\n")
    f.write(f"F1 Score: {avg_f1_score:.4f}\n")

print(f"Metrics saved to {output_filename}")

Metrics saved to ./f1score_output/prompt_aug_sentence_cls/gpt-4o_prompt_aug_sentence_cls_f1score_5039.txt


In [133]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 33 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

In [135]:
prompt_augmentation=True

In [136]:

folder_path = f'./dataset_finalOutput/{model_name}'
os.makedirs(folder_path, exist_ok=True)

if prompt_augmentation==True: 
    dataset_path = f'./dataset_finalOutput/{model_name}/dataset_finalOutput_{model_name}_{judge_model_name}_prompt_aug_sentence_cls_{dataset_len}.csv'
else:
    dataset_path = f'./dataset_finalOutput/{model_name}/dataset_finalOutput_{model_name}_{judge_model_name}_{dataset_len}.csv'
print(dataset_path)

emoji_dataset.to_csv(dataset_path, index=False)

./dataset_finalOutput/gpt-4o/dataset_finalOutput_gpt-4o_gpt-4o-mini_prompt_aug_sentence_cls_5039.csv
